#### New to Plotly?
Plotly's Python library is free and open source! [Get started](https://plot.ly/python/getting-started/) by downloading the client and [reading the primer](https://plot.ly/python/getting-started/).
<br>You can set up Plotly to work in [online](https://plot.ly/python/getting-started/#initialization-for-online-plotting) or [offline](https://plot.ly/python/getting-started/#initialization-for-offline-plotting) mode, or in [jupyter notebooks](https://plot.ly/python/getting-started/#start-plotting-online).
<br>We also have a quick-reference [cheatsheet](https://images.plot.ly/plotly-documentation/images/python_cheat_sheet.pdf) (new!) to help you get started!

#### Mapbox Access Token

To plot on Mapbox maps with Plotly you'll need a Mapbox account and a [Mapbox Access Token](https://www.mapbox.com/studio) which you can add to your [Plotly settings](https://plot.ly/settings/mapbox).

#### Google Maps API
In order to use the `Google Maps - Directions API`, you need to create an account with Google and get your API key [here](https://developers.google.com/maps/documentation/directions/).

In [2]:
import plotly.plotly as py
from plotly.graph_objs import *

import numpy as np
import requests
import copy
import googlemaps

import my_google_maps_api_key

#### Parse Tesla Locations
Perform a `GET` request to retrieve the HTML of the [Google Maps Page](https://www.tesla.com/en_CA/findus#/bounds/70,-50,42,-142,d?search=supercharger,&name=Canada) with all Tesla locations, then parse through to collect all the USA, Canada ones and store them in a dictionary. The dictionary is indexed by `address` and each address has a parameter for `postal_code`, `country`, `latitude` and `longitude`. Be Patient, it takes a while.

In [3]:
r = requests.get('https://www.tesla.com/en_CA/findus#/bounds/70,-50,42,-142,d?search=supercharger,&name=Canada')
r_copy = copy.deepcopy(r.text)
supercharger_locations = {}
valid_countries = ['United States', 'Canada']
params_for_locations = ['postal_code":"', 'country":"', 'latitude":"', 'longitude":"']

while True:
    # add address line to the dictionary
    index = r_copy.find('address_line_1":"')
    if index == -1:
        break
    index += len('address_line_1":"')

    index_end = index
    while r_copy[index_end] != '"':
        index_end += 1
    address_line_1 = r_copy[index:index_end]
    address_line_1 = str(address_line_1)
    supercharger_locations[address_line_1] = {}

    for param in params_for_locations:
        index = r_copy.find(param)
        if index == -1:
            break
        index += len(param)

        index_end = index
        while r_copy[index_end] != '"':
            index_end += 1
        supercharger_locations[address_line_1][param[0:-3]] = r_copy[index:index_end]

    r_copy = r_copy[index_end:len(r.text)]  # slice off the traversed code

all_keys = supercharger_locations.keys()

for key in all_keys:
    if '\\' in supercharger_locations[key] or supercharger_locations[key] == '' or supercharger_locations[key]['postal_code'] == '' or supercharger_locations[key]['country'] not in valid_countries:
        del supercharger_locations[key]

In [4]:
supercharger_locations

{'360 N Rodeo Dr': {'country': u'United States',
  'latitude': u'34.0689030000',
  'longitude': u'-118.4025240000',
  'postal_code': u'90210'},
 '12200 Stemmons Fwy': {'country': u'United States',
  'latitude': u'32.9157160000',
  'longitude': u'-96.8960880000',
  'postal_code': u'75234'},
 '6000 E Camelback Rd': {'country': u'United States',
  'latitude': u'33.5020980000',
  'longitude': u'-111.9519820000',
  'postal_code': u'85251'},
 '16161 Northeast McDougall Road': {'country': u'United States',
  'latitude': u'45.2411500000',
  'longitude': u'-123.0697290000',
  'postal_code': u'97114'},
 '116 Queen St': {'country': u'United States',
  'latitude': u'34.7165180000',
  'longitude': u'-76.6629080000',
  'postal_code': u'28516'},
 '2453 E. Florence Blvd.': {'country': u'United States',
  'latitude': u'32.8787730000',
  'longitude': u'-111.6816940000',
  'postal_code': u'85194'},
 '55 W 14th St': {'country': u'United States',
  'latitude': u'40.7371220000',
  'longitude': u'-73.9961190

#### Table of Locations
Create a table with a sample of the `supercharger_locations` data.

In [5]:
import plotly.plotly as py
from plotly.tools import FigureFactory as FF 

data_matrix = [['Address', 'Country', 'Postal Code', 'Latitude', 'Longitude']]
first_ten_keys = supercharger_locations.keys()[0:10]

for key in first_ten_keys:
    row = [key,
           supercharger_locations[key]['country'],
           supercharger_locations[key]['postal_code'],
           supercharger_locations[key]['latitude'],
           supercharger_locations[key]['longitude']]
    data_matrix.append(row)

table = FF.create_table(data_matrix)
py.iplot(table, filename='supercharger-locations-sample')

#### Plot the Route
The server_key should be replaced with your own Google Maps Directions API key.

In [6]:
def plot_route_between_tesla_stations(address_start, address_end, zoom=3, endpt_size=6):
    start = (supercharger_locations[address_start]['latitude'], supercharger_locations[address_start]['longitude'])
    end = (supercharger_locations[address_end]['latitude'], supercharger_locations[address_end]['longitude'])
    
    directions = gmaps.directions(start, end)
    steps = []
    steps.append(start)  # add starting coordinate to trip
    
    for index in range(len(directions[0]['legs'][0]['steps'])):
        start_coords = directions[0]['legs'][0]['steps'][index]['start_location']
        steps.append((start_coords['lat'], start_coords['lng']))

        if index == len(directions[0]['legs'][0]['steps']) - 1:
            end_coords = directions[0]['legs'][0]['steps'][index]['end_location']
            steps.append((end_coords['lat'], end_coords['lng']))

    steps.append(end)  # add ending coordinate to trip

    mapbox_access_token = 'pk.eyJ1IjoiY2hlbHNlYXBsb3RseSIsImEiOiJjaXFqeXVzdDkwMHFrZnRtOGtlMGtwcGs4In0.SLidkdBMEap9POJGIe1eGw'

    data = Data([
        Scattermapbox(
            lat=[item_x[0] for item_x in steps],
            lon=[item_y[1] for item_y in steps],
            mode='markers+lines',
            marker=Marker(
                size=[endpt_size] + [4 for j in range(len(steps) - 2)] + [endpt_size]
            ),
        )
    ])
    layout = Layout(
        autosize=True,
        hovermode='closest',
        mapbox=dict(
            accesstoken=mapbox_access_token,
            bearing=0,
            style='streets',
            center=dict(
                lat=np.mean([float(step[0]) for step in steps]),
                lon=np.mean([float(step[1]) for step in steps]),
            ),
            pitch=0,
            zoom=zoom
        ),
    )

    fig = dict(data=data, layout=layout)
    return fig

server_key = my_google_maps_api_key.server_key
gmaps = googlemaps.Client(key=server_key)
address_start = '2410 Marine Avenue'
address_end = '1 Rocket Rd'
zoom=12.2
endpt_size=20

fig = plot_route_between_tesla_stations(address_start, address_end, zoom=12.2, endpt_size=20)
py.iplot(fig, filename='tesla-driving-directions-between-superchargers')

In [7]:
from IPython.display import display, HTML

display(HTML('<link href="//fonts.googleapis.com/css?family=Open+Sans:600,400,300,200|Inconsolata|Ubuntu+Mono:400,700" rel="stylesheet" type="text/css" />'))
display(HTML('<link rel="stylesheet" type="text/css" href="http://help.plot.ly/documentation/all_static/css/ipython-notebook-custom.css">'))

! pip install git+https://github.com/plotly/publisher.git --upgrade
import publisher
publisher.publish(
    'tesla-supercharging-stations.ipynb', 'python/tesla-supercharging-stations/', 'Python Tesla Supercharging Stations | Examples | Plotly',
    'How to plot car-travel routes between USA and Canada Telsa Supercharging Stations in Python.',
    title = 'Tesla Supercharging Stations | Plotly',
    name = 'Tesla Supercharging Stations',
    has_thumbnail='true', thumbnail='thumbnail/tesla-stations.jpg',
    language='python', page_type='example_index',
    display_as='maps', order=10)

  Cloning https://github.com/plotly/publisher.git to /var/folders/tc/bs9g6vrd36q74m5t8h9cgphh0000gn/T/pip-f27VSN-build
  Found existing installation: publisher 0.10
    Uninstalling publisher-0.10:
      Successfully uninstalled publisher-0.10
  Running setup.py install for publisher ... - done


/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/IPython/nbconvert.py:13: ShimWarning:

The `IPython.nbconvert` package has been deprecated. You should import from nbconvert instead.

/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/publisher/publisher.py:53: UserWarning:

Did you "Save" this notebook before running this command? Remember to save, always save.

